In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
# Get the current directory
current_directory = os.getcwd()

# Move two levels up (go to the parent directory of the parent directory)
two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

print(two_levels_up_directory)

sys.path.append(two_levels_up_directory)

/Users/lingjiekong/Documents/github/cambioml/pykoi


In [2]:
from milvus import default_server
from pymilvus import connections, utility

import os
import pykoi
import nest_asyncio
nest_asyncio.apply()

/Users/lingjiekong/anaconda3/envs/pykoi-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/lingjiekong/anaconda3/envs/pykoi-test/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["DOC_PATH"] = os.path.join(os.getcwd(), "temp/docs")
os.environ["VECTORDB_PATH"] = os.path.join(os.getcwd(), "temp/vectordb")


with default_server:
    default_server.set_base_dir("{}/milvus".format(os.getenv("VECTORDB_PATH")))
    connections.connect(host="127.0.0.1", port=19530)
    print(utility.get_server_version())
    
    MODEL_SOURCE = "openai"

    #####################################
    # Creating a retrieval QA component #
    #####################################
    # vector database
    vector_db = pykoi.VectorDbFactory.create(
        model_source=MODEL_SOURCE, vector_db_name="milvus")

    # retrieval model with vector database
    retrieval_model = pykoi.RetrievalFactory.create(
        model_source=MODEL_SOURCE, vector_db=vector_db
    )

    # sql database
    database = pykoi.QuestionAnswerDatabase(debug=True)
    dashboard = pykoi.Dashboard(database=database)

    # Creating an OpenAI model
    model = pykoi.ModelFactory.create_model(
        model_source=MODEL_SOURCE, api_key=os.environ["OPENAI_API_KEY"]
    )

    # retrieval and chatbot components
    retriever = pykoi.RetrievalQA(retrieval_model=retrieval_model, vector_db=vector_db)
    chatbot = pykoi.Chatbot(model=model, feedback="vote", is_retrieval=True)

    ############################################################
    # Starting the application and retrieval qa as a component #
    ############################################################
    # Create the application
    app = pykoi.Application(debug=False, share=False)
    app.add_component(retriever)
    app.add_component(chatbot)
    app.add_component(dashboard)
    app.run()

TimeoutError: Milvus not startd in 30.0 seconds